In [2]:
import numpy as np
import keras
from keras import callbacks
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD
from matplotlib import pyplot as plt
from IPython.display import clear_output
from sklearn.metrics import confusion_matrix

Using TensorFlow backend.


In [3]:
img_row = 108
img_col = 108
path = "E:/database108/"
batch_size = 64
output_classes = 24

In [4]:
import random
import pandas as pd
x_train_ori = []
y_train_ori = []
x_test_ori = []
y_test_ori = []
for i in range(516):
    print(i)
    file = pd.read_csv(path + "database_{}.csv".format(i), dtype=str)
    array = file.values
    array = list(array)
    train_test = random.sample(range(100), 60)
    
    for j in train_test[:50]:
        sample = list(array[j])
        lst = []
        for x in sample[1:11665]:
            if type(x) == str:
                x = "".join(x.split())
                lst.append(float(x))
            elif type(x) == float:
                lst.append(x)
            else:
                print(type(x))
        x_train_ori.append(lst)
        y_train_ori.append(sample[0])
        
    for l in train_test[50:]:
        sample = list(array[l])
        lst = []
        for x in sample[1:11665]:
            if type(x) == str:
                x = "".join(x.split())
                lst.append(float(x))
            elif type(x) == float:
                lst.append(x)
            else:
                print(type(x))
        x_test_ori.append(lst)
        y_test_ori.append(sample[0])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [5]:
print(np.shape(x_train_ori))
print(np.shape(x_test_ori))
print(np.shape(y_train_ori))
print(np.shape(y_test_ori))

(25800, 11664)
(5160, 11664)
(25800,)
(5160,)


In [6]:
def y_convert(array, flag=0):
    y = []
    for i in array: # i is the name of each sample:
        name = i.split("_")
        name = name[1:5]
        if flag == 0:
            # category of appearance of a note
            category = np.zeros(24, dtype=int)
            for j in name:
                if j != "x":
                    num = int(j)
                    category[num-60] = 1
            y.append(list(category))
    return y

In [7]:
lst_1 = []
lst_2 = []

for i in y_train_ori:
    label = "".join(i.split())
    lst_1.append(label)
for j in y_test_ori:
    label = "".join(j.split())
    lst_2.append(label)
    
y_train_ori = y_convert(lst_1)
y_test_ori = y_convert(lst_2)

In [8]:
y_train_ori = np.array(y_train_ori)
y_test_ori = np.array(y_test_ori)
x_train_ori = np.array(x_train_ori)
x_test_ori = np.array(x_test_ori)

In [9]:
x_train_ori = x_train_ori.flatten()
x_train_ori = x_train_ori.reshape(25800, img_row, img_col, 1)
x_test_ori = x_test_ori.flatten()
x_test_ori = x_test_ori.reshape(5160, img_row, img_col, 1)

In [10]:
print(x_train_ori.shape)
input_shape = (108, 108, 1)

(25800, 108, 108, 1)


In [11]:
def build_model_A():
    early_stop = callbacks.EarlyStopping(monitor="val_loss", min_delta=0, patience=3, verbose=1, mode="min")
    
    filepath = "Best-weights-my_model-{epoch:03d}-{loss:4f}-{acc:4f}.hdf5"
    checkpoint = callbacks.ModelCheckpoint(filepath, monitor="val_loss", verbose=1, save_best_only= True, mode="min")
    
    callbacks_list = [early_stop, checkpoint]
    
    #build the model
    model = Sequential()
    
    #first convolutional layer
    model.add(Conv2D(32, (3,3), input_shape=input_shape, padding="same", activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    
    #second convolutional layer
    model.add(Conv2D(32, (3,3), activation="relu", border_mode="same"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    
    #third convolutional layer
    model.add(Conv2D(64, (3,3), activation="relu", border_mode = "same"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.4))
    
    #first FC layer
    model.add(Flatten())
    model.add(Dense(256, activation="relu", kernel_initializer="uniform"))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    
    #output layer
    model.add(Dense(24, activation="softmax", kernel_initializer="uniform"))
    
    model.compile(loss="categorical_crossentropy", optimizer="SGD", metrics=["accuracy"])
    return model

In [12]:
# randomly taking 10320 data samples
classifier_A = build_model_A()
hist_A = classifier_A.fit(x_train_ori, y_train_ori, batch_size=128, nb_epoch=50, verbose=1, 
                      validation_data=(x_test_ori, y_test_ori))

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="same", activation="relu")`
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same", activation="relu")`
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 25800 samples, validate on 5160 samples
Epoch 1/50
25800/25800 [==============================] - 18s 703us/step - loss: 8.4349 - acc: 0.3898 - val_loss: 6.5902 - val_acc: 0.3562
Epoch 2/50
25800/25800 [==============================] - 15s 585us/step - loss: 6.6525 - acc: 0.3131 - val_loss: 6.1062 - val_acc: 0.4052
Epoch 3/50
25800/25800 [==============================] - 15s 586us/step - loss: 6.3411 - acc: 0.2949 - val_loss: 5.7046 - val_acc: 0.2731
Epoch 4/50
25800/25800 [==============================] - 15s 585us/step - loss: 6.2017 - acc: 0.2826 - val_loss: 5.8203 - val_acc: 0.1988
Epoch 5/50
25800/25800 [==============================] - 15s 586us/step - loss: 6.1098 - acc: 0.2801 - val_loss: 5.5129 - val_acc: 0.3905
Epoch 6/50
25800/25800 [==============================] - 15s 586us/step - loss: 6.0481 - acc: 0.2753 - val_loss: 5.5198 - val_acc: 0.2151
Epoch 7/50
25800/25800 [==============================] - 15s 588us/step - loss: 6.0056 - acc: 0.2772 - val_loss: 5.4

In [16]:
fname = "weight-25800-CNN_A.hdf5"
classifier_A.save_weights(fname, overwrite=True)

In [13]:
def build_model_B():
    early_stop = callbacks.EarlyStopping(monitor="val_loss", min_delta=0, patience=3, verbose=1, mode="min")
    
    filepath = "Best-weights-my_model-{epoch:03d}-{loss:4f}-{acc:4f}.hdf5"
    checkpoint = callbacks.ModelCheckpoint(filepath, monitor="val_loss", verbose=1, save_best_only= True, mode="min")
    
    callbacks_list = [early_stop, checkpoint]
    
    #build the model
    model = Sequential()
    
    #first convolutional layer
    model.add(Conv2D(32, (3,3), input_shape=input_shape, padding="same", activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    
    #second convolutional layer
    model.add(Conv2D(32, (3,3), activation="relu", border_mode="same"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    
    #third convolutional layer
    model.add(Conv2D(64, (3,3), activation="relu", border_mode = "same"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    
    #forth convolutional layer
    model.add(Conv2D(64, (3,3), activation="relu", border_mode = "same"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.4))
    
    #first FC layer
    model.add(Flatten())
    model.add(Dense(128, activation="relu", kernel_initializer="uniform"))
    model.add(BatchNormalization())
        
    #second FC layer
    model.add(Dense(256, activation="relu", kernel_initializer="uniform"))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    
    #output layer
    model.add(Dense(24, activation="softmax", kernel_initializer="uniform"))
    
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model

In [14]:
# randomly taking 25800 data samples
classifier_B = build_model_B()
hist_B = classifier_B.fit(x_train_ori, y_train_ori, batch_size=64, nb_epoch=40, verbose=1,
                          validation_data=(x_test_ori, y_test_ori))

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="same", activation="relu")`
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same", activation="relu")`
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:28: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same", activation="relu")`
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 25800 samples, validate on 5160 samples
Epoch 1/40
25800/25800 [==============================] - 19s 729us/step - loss: 6.7204 - acc: 0.2876 - val_loss: 5.5816 - val_acc: 0.1934
Epoch 2/40
25800/25800 [==============================] - 17s 675us/step - loss: 5.6990 - acc: 0.2688 - val_loss: 5.2922 - val_acc: 0.2711
Epoch 3/40
25800/25800 [==============================] - 17s 676us/step - loss: 5.5079 - acc: 0.2696 - val_loss: 5.2291 - val_acc: 0.2826
Epoch 4/40
25800/25800 [==============================] - 17s 675us/step - loss: 5.4197 - acc: 0.2674 - val_loss: 5.1835 - val_acc: 0.3240
Epoch 5/40
25800/25800 [==============================] - 17s 676us/step - loss: 5.3545 - acc: 0.2689 - val_loss: 5.1971 - val_acc: 0.2269
Epoch 6/40
25800/25800 [==============================] - 17s 676us/step - loss: 5.3095 - acc: 0.2689 - val_loss: 5.1593 - val_acc: 0.3172
Epoch 7/40
25800/25800 [==============================] - 17s 676us/step - loss: 5.2653 - acc: 0.2674 - val_loss: 5.1

In [15]:
fname = "weight-25800-CNN_B.hdf5"
classifier_B.save_weights(fname, overwrite=True)

In [19]:
def build_model_C():   
    early_stop = callbacks.EarlyStopping(monitor="val_loss", min_delta=0, patience=3, verbose=1, mode="min")
    
    filepath = "Best-weights-my_model-{epoch:03d}-{loss:4f}-{acc:4f}.hdf5"
    checkpoint = callbacks.ModelCheckpoint(filepath, monitor="val_loss", verbose=1, save_best_only= True, mode="min")
    
    callbacks_list = [early_stop, checkpoint]
    
    #build the model
    model = Sequential()
    
    #first convolutional layer
    model.add(Conv2D(32, (3,3), input_shape=input_shape, padding="same", activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    
    #second convolutional layer
    model.add(Conv2D(32, (3,3), activation="relu", border_mode="same"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.2))
    
    #first FC layer
    model.add(Flatten())
    model.add(Dense(256, activation="relu", kernel_initializer="uniform"))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    
    #output layer
    model.add(Dense(24, activation="softmax", kernel_initializer="uniform"))
    
    model.compile(loss="categorical_crossentropy", optimizer="SGD", metrics=["accuracy"])
    return model

In [20]:
# randomly taking 25800 data samples
classifier_C = build_model_C()
hist_C = classifier_C.fit(x_train_ori, y_train_ori, batch_size=64, nb_epoch=40, verbose=1, 
                          validation_data=(x_test_ori, y_test_ori))

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="same", activation="relu")`
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 25800 samples, validate on 5160 samples
Epoch 1/40
25800/25800 [==============================] - 17s 650us/step - loss: 7.1172 - acc: 0.3403 - val_loss: 6.2741 - val_acc: 0.2029
Epoch 2/40
25800/25800 [==============================] - 16s 613us/step - loss: 6.0362 - acc: 0.2770 - val_loss: 5.5431 - val_acc: 0.1828
Epoch 3/40
25800/25800 [==============================] - 16s 614us/step - loss: 5.9119 - acc: 0.2769 - val_loss: 5.4659 - val_acc: 0.2506
Epoch 4/40
25800/25800 [==============================] - 16s 614us/step - loss: 5.8348 - acc: 0.2711 - val_loss: 5.4579 - val_acc: 0.2500
Epoch 5/40
25800/25800 [==============================] - 16s 614us/step - loss: 5.7952 - acc: 0.2726 - val_loss: 5.4811 - val_acc: 0.2072
Epoch 6/40
25800/25800 [==============================] - 16s 614us/step - loss: 5.7616 - acc: 0.2670 - val_loss: 5.4535 - val_acc: 0.3351
Epoch 7/40
25800/25800 [==============================] - 16s 615us/step - loss: 5.7354 - acc: 0.2685 - val_loss: 5.4

In [33]:
fname = "weight-25800-CNN_C.hdf5"
classifier_C.save_weights(fname, overwrite=True)

In [22]:
def build_model_D():
    early_stop = callbacks.EarlyStopping(monitor="val_loss", min_delta=0, patience=3, verbose=1, mode="min")
    
    filepath = "Best-weights-my_model-{epoch:03d}-{loss:4f}-{acc:4f}.hdf5"
    checkpoint = callbacks.ModelCheckpoint(filepath, monitor="val_loss", verbose=1, save_best_only= True, mode="min")
    
    callbacks_list = [early_stop, checkpoint]
    
    #build the model
    model = Sequential()
    
    #first convolutional layer
    model.add(Conv2D(32, (3,3), input_shape=input_shape, padding="same", activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    
    #second convolutional layer
    model.add(Conv2D(32, (3,3), activation="relu", border_mode="same"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    
    #third convolutional layer
    model.add(Conv2D(64, (3,3), activation="relu", border_mode = "same"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    
    #forth convolutional layer
    model.add(Conv2D(64, (3,3), activation="relu", border_mode = "same"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.2))
    
    #first FC layer
    model.add(Flatten())
    model.add(Dense(128, activation="relu", kernel_initializer="uniform"))
    model.add(BatchNormalization())
        
    #second FC layer
    model.add(Dense(128, activation="relu", kernel_initializer="uniform"))
    model.add(BatchNormalization())
    
    #third FC layer
    model.add(Dense(256, activation="relu", kernel_initializer="uniform"))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    
    #output layer
    model.add(Dense(24, activation="softmax", kernel_initializer="uniform"))
    
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model

In [23]:
classifier_D = build_model_D()
hist_D = classifier_D.fit(x_train_ori, y_train_ori, batch_size=32, nb_epoch=40, verbose=1, 
                          validation_data=(x_test_ori, y_test_ori))

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="same", activation="relu")`
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same", activation="relu")`
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:28: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same", activation="relu")`
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 25800 samples, validate on 5160 samples
Epoch 1/40
25800/25800 [==============================] - 23s 894us/step - loss: 6.6334 - acc: 0.2863 - val_loss: 5.4716 - val_acc: 0.2312
Epoch 2/40
25800/25800 [==============================] - 22s 833us/step - loss: 5.6482 - acc: 0.2716 - val_loss: 5.2915 - val_acc: 0.2374
Epoch 3/40
25800/25800 [==============================] - 22s 834us/step - loss: 5.4960 - acc: 0.2683 - val_loss: 5.2423 - val_acc: 0.2845
Epoch 4/40
25800/25800 [==============================] - 21s 833us/step - loss: 5.3981 - acc: 0.2696 - val_loss: 5.2070 - val_acc: 0.2981
Epoch 5/40
25800/25800 [==============================] - 22s 835us/step - loss: 5.3328 - acc: 0.2654 - val_loss: 5.1714 - val_acc: 0.3105
Epoch 6/40
25800/25800 [==============================] - 22s 834us/step - loss: 5.2769 - acc: 0.2626 - val_loss: 5.1535 - val_acc: 0.3494
Epoch 7/40
25800/25800 [==============================] - 22s 834us/step - loss: 5.2466 - acc: 0.2690 - val_loss: 5.1

In [34]:
fname = "weight-25800-CNN_D.hdf5"
classifier_D.save_weights(fname, overwrite=True)

In [25]:
classifier_B200 = build_model_B()
hist_B200 = classifier_B200.fit(x_train_ori, y_train_ori, batch_size=32, nb_epoch=200, verbose=1, 
                          validation_data=(x_test_ori, y_test_ori))

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="same", activation="relu")`
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same", activation="relu")`
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:28: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same", activation="relu")`
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 25800 samples, validate on 5160 samples
Epoch 1/200
25800/25800 [==============================] - 22s 869us/step - loss: 6.5958 - acc: 0.2880 - val_loss: 5.4642 - val_acc: 0.2196
Epoch 2/200
25800/25800 [==============================] - 21s 802us/step - loss: 5.6624 - acc: 0.2705 - val_loss: 5.2693 - val_acc: 0.1919
Epoch 3/200
25800/25800 [==============================] - 21s 804us/step - loss: 5.4873 - acc: 0.2710 - val_loss: 5.2168 - val_acc: 0.2242
Epoch 4/200
25800/25800 [==============================] - 21s 805us/step - loss: 5.3961 - acc: 0.2667 - val_loss: 5.1821 - val_acc: 0.2198
Epoch 5/200
25800/25800 [==============================] - 21s 805us/step - loss: 5.3269 - acc: 0.2678 - val_loss: 5.1655 - val_acc: 0.3471
Epoch 6/200
25800/25800 [==============================] - 21s 805us/step - loss: 5.2831 - acc: 0.2677 - val_loss: 5.1456 - val_acc: 0.2266
Epoch 7/200
25800/25800 [==============================] - 21s 803us/step - loss: 5.2505 - acc: 0.2633 - val_lo

Epoch 59/200
25800/25800 [==============================] - 21s 804us/step - loss: 5.1659 - acc: 0.2660 - val_loss: 5.1209 - val_acc: 0.3397
Epoch 60/200
25800/25800 [==============================] - 21s 805us/step - loss: 5.1648 - acc: 0.2704 - val_loss: 5.1183 - val_acc: 0.2469
Epoch 61/200
25800/25800 [==============================] - 21s 805us/step - loss: 5.1632 - acc: 0.2698 - val_loss: 5.1185 - val_acc: 0.1393
Epoch 62/200
25800/25800 [==============================] - 21s 805us/step - loss: 5.1638 - acc: 0.2695 - val_loss: 5.1169 - val_acc: 0.2529
Epoch 63/200
25800/25800 [==============================] - 21s 806us/step - loss: 5.1632 - acc: 0.2684 - val_loss: 5.1176 - val_acc: 0.2899
Epoch 64/200
25800/25800 [==============================] - 21s 806us/step - loss: 5.1641 - acc: 0.2673 - val_loss: 5.1209 - val_acc: 0.2068
Epoch 65/200
25800/25800 [==============================] - 21s 806us/step - loss: 5.1636 - acc: 0.2721 - val_loss: 5.1195 - val_acc: 0.3624
Epoch 66/200


Epoch 117/200
25800/25800 [==============================] - 21s 804us/step - loss: 5.1524 - acc: 0.2665 - val_loss: 5.1158 - val_acc: 0.4095
Epoch 118/200
25800/25800 [==============================] - 21s 804us/step - loss: 5.1537 - acc: 0.2675 - val_loss: 5.1186 - val_acc: 0.2508
Epoch 119/200
25800/25800 [==============================] - 21s 804us/step - loss: 5.1546 - acc: 0.2679 - val_loss: 5.1167 - val_acc: 0.2446
Epoch 120/200
25800/25800 [==============================] - 21s 805us/step - loss: 5.1542 - acc: 0.2692 - val_loss: 5.1135 - val_acc: 0.2076
Epoch 121/200
25800/25800 [==============================] - 21s 805us/step - loss: 5.1540 - acc: 0.2694 - val_loss: 5.1140 - val_acc: 0.3634
Epoch 122/200
25800/25800 [==============================] - 21s 804us/step - loss: 5.1537 - acc: 0.2683 - val_loss: 5.1148 - val_acc: 0.2008
Epoch 123/200
25800/25800 [==============================] - 21s 805us/step - loss: 5.1515 - acc: 0.2686 - val_loss: 5.1136 - val_acc: 0.2568
Epoch 

Epoch 175/200
25800/25800 [==============================] - 21s 804us/step - loss: 5.1485 - acc: 0.2683 - val_loss: 5.1125 - val_acc: 0.1477
Epoch 176/200
25800/25800 [==============================] - 21s 804us/step - loss: 5.1487 - acc: 0.2719 - val_loss: 5.1106 - val_acc: 0.2868
Epoch 177/200
25800/25800 [==============================] - 21s 805us/step - loss: 5.1472 - acc: 0.2667 - val_loss: 5.1146 - val_acc: 0.2114
Epoch 178/200
25800/25800 [==============================] - 21s 806us/step - loss: 5.1476 - acc: 0.2666 - val_loss: 5.1122 - val_acc: 0.2112
Epoch 179/200
25800/25800 [==============================] - 21s 805us/step - loss: 5.1486 - acc: 0.2734 - val_loss: 5.1122 - val_acc: 0.2318
Epoch 180/200
25800/25800 [==============================] - 21s 805us/step - loss: 5.1487 - acc: 0.2692 - val_loss: 5.1156 - val_acc: 0.1417
Epoch 181/200
25800/25800 [==============================] - 21s 806us/step - loss: 5.1476 - acc: 0.2642 - val_loss: 5.1107 - val_acc: 0.2802
Epoch 

In [1]:
def build_model_E():
    early_stop = callbacks.EarlyStopping(monitor="val_loss", min_delta=0, patience=3, verbose=1, mode="min")
    
    filepath = "Best-weights-my_model-{epoch:03d}-{loss:4f}-{acc:4f}.hdf5"
    checkpoint = callbacks.ModelCheckpoint(filepath, monitor="val_loss", verbose=1, save_best_only= True, mode="min")
    
    callbacks_list = [early_stop, checkpoint]
    
    #build the model
    model = Sequential()
    
    #first convolutional layer
    model.add(Conv2D(32, (3,3), input_shape=input_shape, padding="same", activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    
    #second convolutional layer
    model.add(Conv2D(32, (3,3), activation="relu", border_mode="same"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    
    #third convolutional layer
    model.add(Conv2D(64, (3,3), activation="relu", border_mode = "same"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))

    #forth convolutional layer
    model.add(Conv2D(64, (3,3), activation="relu", border_mode = "same"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    
    #fifth convulutional layer
    model.add(Conv2D(64, (3,3), activation="relu", border_mode = "same"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.3))
    
    #first FC layer
    model.add(Flatten())
    model.add(Dense(64, activation="relu", kernel_initializer="uniform"))
    model.add(BatchNormalization())
        
    #second FC layer
    model.add(Dense(128, activation="relu", kernel_initializer="uniform"))
    model.add(BatchNormalization())
    
    #third FC layer
    model.add(Dense(256, activation="relu", kernel_initializer="uniform"))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    
    #output layer
    model.add(Dense(24, activation="sigmoid", kernel_initializer="uniform"))
    
    model.compile(loss="mean_squared_error", optimizer="adam", metrics=["accuracy"])
    return model

In [35]:
fname = "weight-25800-CNN_B200.hdf5"
classifier_B200.save_weights(fname, overwrite=True)

In [28]:
classifier_E = build_model_E()
hist_E = classifier_E.fit(x_train_ori, y_train_ori, batch_size=32, nb_epoch=100, verbose=1, 
                          validation_data=(x_test_ori, y_test_ori))

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="same", activation="relu")`
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same", activation="relu")`
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:28: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same", activation="relu")`
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:33: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same", activation="relu")`
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_ep

Train on 25800 samples, validate on 5160 samples
Epoch 1/100
25800/25800 [==============================] - 24s 945us/step - loss: 7.2339 - acc: 0.2927 - val_loss: 5.5839 - val_acc: 0.3688
Epoch 2/100
25800/25800 [==============================] - 23s 873us/step - loss: 5.8427 - acc: 0.2774 - val_loss: 5.3288 - val_acc: 0.2688
Epoch 3/100
25800/25800 [==============================] - 23s 873us/step - loss: 5.6262 - acc: 0.2711 - val_loss: 5.2607 - val_acc: 0.3196
Epoch 4/100
25800/25800 [==============================] - 23s 883us/step - loss: 5.5094 - acc: 0.2715 - val_loss: 5.2229 - val_acc: 0.2463
Epoch 5/100
25800/25800 [==============================] - 22s 871us/step - loss: 5.4241 - acc: 0.2664 - val_loss: 5.1873 - val_acc: 0.2008
Epoch 6/100
25800/25800 [==============================] - 22s 865us/step - loss: 5.3565 - acc: 0.2658 - val_loss: 5.2711 - val_acc: 0.2376
Epoch 7/100
25800/25800 [==============================] - 22s 864us/step - loss: 5.3178 - acc: 0.2686 - val_lo

Epoch 59/100
25800/25800 [==============================] - 22s 861us/step - loss: 5.1832 - acc: 0.2687 - val_loss: 5.1205 - val_acc: 0.2791
Epoch 60/100
25800/25800 [==============================] - 22s 862us/step - loss: 5.1823 - acc: 0.2669 - val_loss: 5.1257 - val_acc: 0.1436
Epoch 61/100
25800/25800 [==============================] - 22s 864us/step - loss: 5.1774 - acc: 0.2664 - val_loss: 5.1239 - val_acc: 0.2940
Epoch 62/100
25800/25800 [==============================] - 22s 863us/step - loss: 5.1763 - acc: 0.2676 - val_loss: 5.1172 - val_acc: 0.3345
Epoch 63/100
25800/25800 [==============================] - 22s 863us/step - loss: 5.1739 - acc: 0.2690 - val_loss: 5.1180 - val_acc: 0.2512
Epoch 64/100
25800/25800 [==============================] - 22s 863us/step - loss: 5.1801 - acc: 0.2664 - val_loss: 5.1188 - val_acc: 0.3029
Epoch 65/100
25800/25800 [==============================] - 22s 863us/step - loss: 5.1799 - acc: 0.2653 - val_loss: 5.1276 - val_acc: 0.0901
Epoch 66/100


In [39]:
fname = "weight-25800-CNN_E.hdf5"
classifier_E.save_weights(fname, overwrite=True)

In [30]:
def build_model_F():
    early_stop = callbacks.EarlyStopping(monitor="val_loss", min_delta=0, patience=3, verbose=1, mode="min")
    
    filepath = "Best-weights-my_model-{epoch:03d}-{loss:4f}-{acc:4f}.hdf5"
    checkpoint = callbacks.ModelCheckpoint(filepath, monitor="val_loss", verbose=1, save_best_only= True, mode="min")
    
    callbacks_list = [early_stop, checkpoint]
    
    #build the model
    model = Sequential()
    
    #first convolutional layer
    model.add(Conv2D(32, (3,3), input_shape=input_shape, padding="same", activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    
    #second convolutional layer
    model.add(Conv2D(32, (3,3), activation="relu", border_mode="same"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    
    #third convolutional layer
    model.add(Conv2D(32, (3,3), activation="relu", border_mode = "same"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))

    #forth convolutional layer
    model.add(Conv2D(64, (3,3), activation="relu", border_mode = "same"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    
    #fifth convolutional layer
    model.add(Conv2D(64, (3,3), activation="relu", border_mode = "same"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    
    #sixth convolutional layer
    model.add(Conv2D(64, (3,3), activation="relu", border_mode = "same"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.4))
    
    #first FC layer
    model.add(Flatten())
    model.add(Dense(64, activation="relu", kernel_initializer="uniform"))
    model.add(BatchNormalization())
        
    #second FC layer
    model.add(Dense(128, activation="relu", kernel_initializer="uniform"))
    model.add(BatchNormalization())
    
    #third FC layer
    model.add(Dense(256, activation="relu", kernel_initializer="uniform"))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    
    #output layer
    model.add(Dense(24, activation="softmax", kernel_initializer="uniform"))
    
    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model

In [32]:
classifier_F = build_model_F()
hist_F = classifier_F.fit(x_train_ori, y_train_ori, batch_size=32, nb_epoch=50, verbose=1, 
                          validation_data=(x_test_ori, y_test_ori))

c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="same", activation="relu")`
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), padding="same", activation="relu")`
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:28: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same", activation="relu")`
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:33: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same", activation="relu")`
c:\users\zheng\anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:38: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same", 

Train on 25800 samples, validate on 5160 samples
Epoch 1/50
25800/25800 [==============================] - 25s 985us/step - loss: 8.7690 - acc: 0.3352 - val_loss: 6.3610 - val_acc: 0.3043
Epoch 2/50
25800/25800 [==============================] - 23s 887us/step - loss: 6.8167 - acc: 0.2984 - val_loss: 5.6635 - val_acc: 0.2690
Epoch 3/50
25800/25800 [==============================] - 23s 887us/step - loss: 6.3542 - acc: 0.2852 - val_loss: 5.4263 - val_acc: 0.2556
Epoch 4/50
25800/25800 [==============================] - 23s 879us/step - loss: 6.1190 - acc: 0.2832 - val_loss: 5.3327 - val_acc: 0.2215
Epoch 5/50
25800/25800 [==============================] - 22s 871us/step - loss: 5.9770 - acc: 0.2760 - val_loss: 5.2747 - val_acc: 0.3746
Epoch 6/50
25800/25800 [==============================] - 22s 868us/step - loss: 5.8715 - acc: 0.2736 - val_loss: 5.2249 - val_acc: 0.2748
Epoch 7/50
25800/25800 [==============================] - 22s 872us/step - loss: 5.7875 - acc: 0.2681 - val_loss: 5.2

In [36]:
fname = "weight-25800-CNN_F.hdf5"
classifier_F.save_weights(fname, overwrite=True)